In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.tools import tool
#from langchain_community.tools import DuckDuckGoSearchAPIWrapper
#from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.llms import Ollama

# Initialize local LLMs using Ollama
research_llm = Ollama(model="deepseek-r1")  # Use DeepSeek R1 for research tasks
code_llm = Ollama(model="deepseek-coder")  # Use DeepSeek Coder for coding tasks

# Custom Tools (if needed)
class CustomTools:
    @tool("Save to File")
    def save_to_file(content: str, filename: str):
        """Saves content to a file."""
        with open(filename, "w") as file:
            file.write(content)
        return f"Content saved to {filename}"

# Initialize tools
search_tool = GoogleSerperAPIWrapper()  # Free search API
custom_tools = CustomTools()

# Define Agents
research_agent = Agent(
    role='Senior Research Analyst',
    goal='Find and analyze the most impactful academic papers',
    backstory="""You're a renowned research analyst with a knack for finding 
    cutting-edge papers in any field. You have access to all major academic 
    databases and can quickly assess paper quality.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool.run],  # Use DuckDuckGo search tool
    llm=research_llm  # Use DeepSeek R1 for research
)

code_agent = Agent(
    role='Senior Software Engineer',
    goal='Generate high-quality code implementations from research methodologies',
    backstory="""You're a principal engineer at a top AI lab who specializes 
    in translating research papers into working implementations.""",
    verbose=True,
    allow_delegation=False,
    tools=[custom_tools.save_to_file],
    llm=code_llm  # Use DeepSeek Coder for coding
)

# Define Tasks
research_task = Task(
    description="""Find the best paper on {topic}. Extract its methodology section 
    and provide a detailed explanation of the approach.""",
    expected_output="A detailed methodology section from the best paper on the topic.",
    agent=research_agent,
    output_file='methodology.txt'
)

coding_task = Task(
    description="""Take the methodology from the research agent and generate Python code 
    that implements the described approach. Include test cases and documentation.""",
    expected_output="Complete Python implementation with example usage.",
    agent=code_agent,
    output_file='implementation.py'
)

# Create Crew
implementation_crew = Crew(
    agents=[research_agent, code_agent],
    tasks=[research_task, coding_task],
    verbose=2,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = implementation_crew.kickoff(inputs={'topic': topic})
    print("\n\n########################")
    print("## Workflow Complete ##")
    print("########################")
    print(result)

C:\Users\samra\AppData\Local\Temp\ipykernel_5116\121158271.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  research_llm = Ollama(model="deepseek-r1")  # Use DeepSeek R1 for research tasks


ValidationError: 1 validation error for Agent
tools
  Value error, Invalid tool type: <class 'langchain_community.utilities.duckduckgo_search.DuckDuckGoSearchAPIWrapper'>. Tool must be an instance of BaseTool or an object with 'name', 'func', and 'description' attributes. [type=value_error, input_value=[DuckDuckGoSearchAPIWrapp...='auto', source='text')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [10]:
import os
import requests
#import spacy
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import GoogleSerperAPIWrapper
#from langchain_community.llms import Ollama
#from langchain.llms import Ollama
from langchain_core.tools import Tool

# Load spaCy model for NLP
#nlp = spacy.load("en_core_web_sm")

# Initialize local LLM for research tasks
#OLLAMA_URL = "http://127.0.0.1:12345"
#research_llm = Ollama(model="deepseek-r1:latest",base_url=OLLAMA_URL)  # Use DeepSeek R1 for research
from langchain_community.llms import Ollama

OLLAMA_URL = "http://127.0.0.1:12345"
research_llm = Ollama(
    model="deepseek-r1",
    base_url=OLLAMA_URL,
    litellm_provider="ollama"  # Explicitly define the provider
)


# Initialize search tool using Google Serper API
search_tool = GoogleSerperAPIWrapper(serper_api_key="8a8810d7af80d6ef588cdd12b9e913ca20baa4c0")
search_tool_wrapped = Tool(
    name="Google Serper Search",
    func=search_tool.run,
    description="Searches Google for relevant academic papers"
)
# Define Research Agent
research_agent = Agent(
    role='Senior Research Analyst',
    goal='Find and analyze the most impactful academic papers',
    backstory="""You're a renowned research analyst with a knack for finding 
    cutting-edge papers in any field. You have access to all major academic 
    databases and can quickly assess paper quality.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool_wrapped],
    llm=research_llm  # Use DeepSeek R1 for research
)

# Define Research Task
research_task = Task(
    description="Find the best paper on {topic} using Google Serper API.",
    expected_output="A list of relevant research papers on the topic.",
    agent=research_agent,
    output_file='search_results.txt'
)

# Create Crew
search_crew = Crew(
    agents=[research_agent],
    tasks=[research_task],
    verbose=True,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = search_crew.kickoff(inputs={'topic': topic})
    print("\n\n########################")
    print("## Search Complete ##")
    print("########################")
    print(result)


ValidationError: 1 validation error for Ollama
litellm_provider
  Extra inputs are not permitted [type=extra_forbidden, input_value='ollama', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [11]:
from langchain_community.llms import Ollama

OLLAMA_URL = "http://127.0.0.1:12345"
llm = Ollama(model="deepseek-r1", base_url=OLLAMA_URL)

response = llm.invoke("Explain machine learning in simple terms.")
print(response)


<think>
Okay, so I need to explain machine learning in simple terms. Hmm, where do I start? Well, maybe first think about what machine learning is all about. From what I understand, it's a way for computers to learn from data without being explicitly programmed. But how exactly does that work?

Maybe I should break it down into simpler concepts. Like, when you teach a child to recognize objects, they see many examples and then can identify them. Is machine learning similar? Instead of a teacher, the computer learns by analyzing patterns in data.

What are some common applications? Well, recommendation systems on Netflix or Spotify probably use ML to suggest shows or songs based on your past choices. Maybe facial recognition is another example; it's used in security or apps like Instagram's filters.

So, how does the process actually happen? I think it involves feeding a lot of data into an algorithm, which then finds patterns within that data. Once the model learns from this data, it c

In [12]:
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.llms import Ollama
from langchain_core.tools import Tool

# Initialize Ollama for LLM processing
OLLAMA_URL = "http://127.0.0.1:12345"
research_llm = Ollama(model="deepseek-r1", base_url=OLLAMA_URL)  # Ensure the model exists

# Initialize Google Serper API tool
search_tool = GoogleSerperAPIWrapper(serper_api_key="8a8810d7af80d6ef588cdd12b9e913ca20baa4c0")
search_tool_wrapped = Tool(
    name="Google Serper Search",
    func=search_tool.run,
    description="Searches Google for relevant academic papers"
)

# Define Research Agent with LLM
research_agent = Agent(
    role="Senior Research Analyst",
    goal="Find and analyze the most impactful academic papers",
    backstory="You're a research expert who specializes in finding top papers in any field.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool_wrapped],
    llm=research_llm  # ✅ Ensure Ollama is used properly
)

# Define Research Task
research_task = Task(
    description="Find the best paper on {topic} using Google Serper API.",
    expected_output="A list of relevant research papers on the topic.",
    agent=research_agent,
    output_file="search_results.txt"
)

# Create Crew
search_crew = Crew(
    agents=[research_agent],
    tasks=[research_task],
    verbose=True,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = search_crew.kickoff(inputs={"topic": topic})

    print("\n\n########################")
    print("## Search Complete ##")
    print("########################")
    print(result)


ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



Enter research topic:  cwt


ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Ollama
Params: {'model': 'deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Research Analyst
## Task: Find the best paper on cwt using Google Serper API.

Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Ollama
Params: {'model': 'deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers


BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=[1mOllama[0m
Params: {'model': 'deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [13]:
response = research_llm.invoke("Test query")
print(response)


<think>

</think>

It seems like you might be testing or checking something. How can I assist you? If you need information, a tool, or have a question, feel free to let me know! 😊


In [14]:
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.llms import Ollama
from langchain_core.tools import Tool

# Initialize Ollama for LLM processing
OLLAMA_URL = "http://127.0.0.1:12345"
research_llm = Ollama(
    model="ollama/deepseek-r1",  # Explicitly specify the provider
    base_url=OLLAMA_URL
)

# Initialize Google Serper API tool
search_tool = GoogleSerperAPIWrapper(serper_api_key="8a8810d7af80d6ef588cdd12b9e913ca20baa4c0")
search_tool_wrapped = Tool(
    name="Google Serper Search",
    func=search_tool.run,
    description="Searches Google for relevant academic papers"
)

# Define Research Agent with LLM
research_agent = Agent(
    role="Senior Research Analyst",
    goal="Find and analyze the most impactful academic papers",
    backstory="You're a research expert who specializes in finding top papers in any field.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool_wrapped],
    llm=research_llm  # ✅ Ensure Ollama is used properly
)

# Define Research Task
research_task = Task(
    description="Find the best paper on {topic} using Google Serper API.",
    expected_output="A list of relevant research papers on the topic.",
    agent=research_agent,
    output_file="search_results.txt"
)

# Create Crew
search_crew = Crew(
    agents=[research_agent],
    tasks=[research_task],
    verbose=True,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = search_crew.kickoff(inputs={"topic": topic})

    print("\n\n########################")
    print("## Search Complete ##")
    print("########################")
    print(result)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



Enter research topic:  cow


ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Ollama
Params: {'model': 'ollama/deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Research Analyst
## Task: Find the best paper on cow using Google Serper API.

Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Ollama
Params: {'model': 'ollama/deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers


BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=[1mOllama[0m
Params: {'model': 'ollama/deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [15]:
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.llms import Ollama
from langchain_core.tools import Tool

# Initialize Ollama for LLM processing
OLLAMA_URL = "http://127.0.0.1:12345"
research_llm = Ollama(
    model="ollama/deepseek-r1",  # Explicitly specify the provider
    base_url=OLLAMA_URL
)

# Initialize Google Serper API tool
search_tool = GoogleSerperAPIWrapper(serper_api_key="8a8810d7af80d6ef588cdd12b9e913ca20baa4c0")
search_tool_wrapped = Tool(
    name="Google Serper Search",
    func=search_tool.run,
    description="Searches Google for relevant academic papers"
)

# Define Research Agent with LLM
research_agent = Agent(
    role="Senior Research Analyst",
    goal="Find and analyze the most impactful academic papers",
    backstory="You're a research expert who specializes in finding top papers in any field.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool_wrapped],
    llm=research_llm  # ✅ Ensure Ollama is used properly
)

# Define Research Task
research_task = Task(
    description="Find the best paper on {topic} using Google Serper API.",
    expected_output="A list of relevant research papers on the topic.",
    agent=research_agent,
    output_file="search_results.txt"
)

# Create Crew
search_crew = Crew(
    agents=[research_agent],
    tasks=[research_task],
    verbose=True,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = search_crew.kickoff(inputs={"topic": topic})

    print("\n\n########################")
    print("## Search Complete ##")
    print("########################")
    print(result)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



Enter research topic:  cow


ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Ollama
Params: {'model': 'ollama/deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Research Analyst
## Task: Find the best paper on cow using Google Serper API.

Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Ollama
Params: {'model': 'ollama/deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers


BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=[1mOllama[0m
Params: {'model': 'ollama/deepseek-r1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [16]:
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool

# Initialize Custom Ollama LLM
research_llm = CustomOllamaLLM(base_url="http://127.0.0.1:12345", model="deepseek-r1")

# Initialize Google Serper API tool
search_tool = GoogleSerperAPIWrapper(serper_api_key="8a8810d7af80d6ef588cdd12b9e913ca20baa4c0")
search_tool_wrapped = Tool(
    name="Google Serper Search",
    func=search_tool.run,
    description="Searches Google for relevant academic papers"
)

# Define Research Agent with Custom LLM
research_agent = Agent(
    role="Senior Research Analyst",
    goal="Find and analyze the most impactful academic papers",
    backstory="You're a research expert who specializes in finding top papers in any field.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool_wrapped],
    llm=research_llm  # Use the custom Ollama LLM
)

# Define Research Task
research_task = Task(
    description="Find the best paper on {topic} using Google Serper API.",
    expected_output="A list of relevant research papers on the topic.",
    agent=research_agent,
    output_file="search_results.txt"
)

# Create Crew
search_crew = Crew(
    agents=[research_agent],
    tasks=[research_task],
    verbose=True,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = search_crew.kickoff(inputs={"topic": topic})

    print("\n\n########################")
    print("## Search Complete ##")
    print("########################")
    print(result)

NameError: name 'CustomOllamaLLM' is not defined